In [1]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os

NumOfMice = "10031"
totalpath = "G:\YSY"
maze_type = 2

# 需要从G:\YSY读取pickle文件

In [2]:
#from scipy.io import loadmat
import h5py
import numpy as np

#读入Cell_Reg文件
f = h5py.File("cellRegistered.mat",'r')
ms_reg = f['cell_registered_struct']
index_map = np.array(ms_reg['cell_to_index_map'])

In [3]:
# index map 统计与排序
import copy as cp

def NumOfDaysRecorded(index_map):
    RecordVec = []
    for i in range(np.shape(index_map)[1]):
        sum = 0
        for j in range(np.shape(index_map)[0]):
            if index_map[j][i] != 0:
                sum = sum + 1
        RecordVec = RecordVec + [sum]
    return RecordVec

RecordVec = NumOfDaysRecorded(index_map)
print("RecordVec:  ",'\n',RecordVec, end='\n\n')
print("After rearrangement...", end='\n\n')

def partition(index_map, RecordVec, low, high): 
    i = low - 1         # 最小元素索引
    pivot = RecordVec[high]       
    for j in range(low, high): 
        # 当前元素大于或等于 pivot 
        if   RecordVec[j] > pivot:           
            i = i+1
            RecordVec[i], RecordVec[j] = RecordVec[j], RecordVec[i]
            c = cp.deepcopy(index_map[:,i])
            index_map[:, i] = cp.deepcopy(index_map[:, j])
            index_map[:, j] = cp.deepcopy(c)      
            
    RecordVec[i+1], RecordVec[high] = RecordVec[high], RecordVec[i+1]
    c = cp.deepcopy(index_map[:,i+1])
    index_map[:, i+1] = cp.deepcopy(index_map[:, high])
    index_map[:, high] = cp.deepcopy(c)
    return ( i+1 ) 
    
def quickSort(index_map, RecordVec, low, high): 
    if low < high:   
        pi = partition(index_map, RecordVec, low, high)
        quickSort(index_map, RecordVec, low, pi-1)
        quickSort(index_map, RecordVec, pi+1, high)

quickSort(index_map, RecordVec, 0, len(RecordVec)-1)

RecordVec:   
 [8, 7, 8, 1, 4, 6, 6, 1, 6, 6, 5, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 7, 6, 7, 6, 8, 7, 7, 8, 7, 5, 1, 7, 2, 3, 1, 5, 5, 1, 3, 3, 5, 2, 5, 4, 6, 1, 7, 3, 6, 2, 7, 8, 6, 1, 6, 2, 4, 4, 2, 2, 1, 2, 4, 1, 6, 8, 6, 7, 3, 8, 3, 4, 6, 6, 6, 5, 3, 8, 4, 4, 3, 5, 6, 6, 2, 3, 3, 1, 2, 1, 4, 1, 1, 7, 1, 1, 4, 4, 5, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 3, 5, 3, 6, 2, 4, 6, 1, 3, 2, 2, 7, 1, 1, 1, 1, 6, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 6, 4, 4, 2, 2, 6, 1, 1, 6, 1, 3, 6, 5, 7, 2, 4, 4, 2, 1, 1, 4, 2, 2, 2, 2, 3, 1, 2, 1, 2, 1, 2, 1]

After rearrangement...



In [4]:
dateset = ["20200428",
           "20200429",
           "20200430",
           "20200501",
           "20200502",
           "20200503",
           "20200504",
           "20200505"]

In [5]:
import pylab as pl

def plot_ratemap(ratemap, axes=None, title=None, extent = [0 , 12, 0 , 12],title_color = "black", *args, **kwargs):  # pylint: disable=keyword-arg-before-vararg
    """Plot ratemaps."""
    if axes is None:
        axes = pl.gca()
    # Plot the ratemap
    axes.imshow(ratemap, interpolation='none', extent = extent, origin = "lower", alpha = 0.8, *args, **kwargs, cmap = 'copper')
    # ax.pcolormesh(ratemap, *args, **kwargs)
    axes.axis('off')
    if title is not None:
        axes.set_title(title, color = title_color)

In [6]:
import operator

for i in range(8):
    with open(os.path.join(totalpath,NumOfMice,"Days_aligned","maze"+str(maze_type),"rate_map_all_"+dateset[i]+".pkl"), 'rb') as handle:
        if i == 0:
            [is_placecell_isi, SI_all, rate_map_all] = pickle.load(handle)
            f = [[is_placecell_isi, SI_all, rate_map_all]]
        else:
            [is_placecell_isi, SI_all, rate_map_all] = pickle.load(handle)
            f.append([is_placecell_isi, SI_all, rate_map_all])
for i in range(8):
    for j in range(3):
        print(np.shape(f[i][j]))
    print(max(index_map[i]))
    print()

(120,)
(120,)
(120, 144)
120.0

(94,)
(94,)
(94, 144)
94.0

(88,)
(88,)
(88, 144)
88.0

(91,)
(91,)
(91, 144)
91.0

(73,)
(73,)
(73, 144)
73.0

(89,)
(89,)
(89, 144)
89.0

(72,)
(72,)
(72, 144)
72.0

(83,)
(83,)
(83, 144)
83.0



In [7]:
# plot days align
import matplotlib.pyplot as plt
n_neuron = 183
cols = 8
rows = n_neuron
fig = plt.figure(figsize=(24, rows * 4))
for i in range(n_neuron):
    for d in range(len(dateset)):
        if index_map[d,i] == 0:
            continue
        index = index_map[d,i] - 1
        rf = plt.subplot(rows , cols, 8*i + d + 1)
        title = "%d %s (%.2f)" % (index, "day "+str(d+1), f[d][1][int(index)])
        if f[d][0][int(index)] == 1:
            plot_ratemap(np.reshape(f[d][2][int(index),], [12,12]), axes=rf, title=title, title_color = "red")
        else:
            plot_ratemap(np.reshape(f[d][2][int(index),], [12,12]), axes=rf, title=title)
        rf.invert_yaxis()

path = os.path.join(totalpath,NumOfMice,"Days_aligned","maze"+str(maze_type),'aligned_rate_map.pdf')
plt.savefig(path, format="pdf")
path = os.path.join(totalpath,NumOfMice,"Days_aligned","maze"+str(maze_type),'aligned_rate_map.png')
plt.savefig(path, format="png")